### ResNet

In [ ]:
import numpy as np
import jax
import jax.numpy as jnp
import haiku as hk
import optax

#### import CIFAR-10

In [ ]:
import torch
import torchvision

PATH = 'data'
BATCH_SIZE = 16


def custom_transform(x):
    return (np.array(x, dtype=np.float32)/255.0 -0.5) * 2

def custom_collate_fn(batch):
    """ gets list of tuples and returns seperated images and labels as ndarrays """
    transposed_data = list(zip(*batch))

    labels = np.array(transposed_data[1])
    imgs = np.stack(transposed_data[0])

    return imgs, labels


train_data = torchvision.datasets.CIFAR10(root=PATH, train=True, transform=custom_transform, download=True)
test_data = torchvision.datasets.CIFAR10(root=PATH, train=False, transform=custom_transform, download=True)


train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=custom_collate_fn)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, collate_fn=custom_collate_fn)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

#### check data

In [ ]:
import matplotlib.pyplot as plt

x, y = next(iter(train_loader))

print('x[0].shape: ', x[0].shape)

img = x[0] * 0.5 + 0.5

print(f'label: {classes[y[0]]}, {y[0]}')
plt.imshow(img)
plt.axis("off")

#### define model

In [ ]:

class Res1(hk.Module):
    """Residual Module 1"""
    def __init__(self):
        super().__init__()
        
    def __call__(self, x_in, is_training):
        
        x = hk.Conv2D(output_channels=32, kernel_shape=(3,3), padding="SAME")(x_in)
        x = hk.BatchNorm(create_scale=False, create_offset=False, decay_rate=0.9)(x, is_training=is_training)
        x = jax.nn.relu(x)
        x = hk.Conv2D(output_channels=32, kernel_shape=(3,3), padding="SAME")(x)
        x = hk.BatchNorm(create_scale=False, create_offset=False, decay_rate=0.9)(x, is_training=is_training)
        x = jax.nn.relu(x)
        
        return x
    
class Res2(hk.Module):
    """Residual Module 2"""
    def __init__(self):
        super().__init__()
        
    def __call__(self, x_in, is_training):
        
        x = hk.Conv2D(output_channels=64, kernel_shape=(3,3), stride=2, padding="SAME")(x_in)
        x = hk.BatchNorm(create_scale=False, create_offset=False, decay_rate=0.9)(x, is_training=is_training)
        x = jax.nn.relu(x)
        x = hk.Conv2D(output_channels=64, kernel_shape=(3,3), padding="SAME")(x)
        x = hk.BatchNorm(create_scale=False, create_offset=False, decay_rate=0.9)(x, is_training=is_training)
        x = jax.nn.relu(x)
        
        return x
    
class ResNet(hk.Module):
    """ResNet"""
    def __init__(self):
        super().__init__()

    def __call__(self, x_in, is_training):
        
        x_res1 = Res1()(x_in, is_training)
        x = hk.Conv2D(output_channels=32, kernel_shape=1)(x_in)
        x = x + x_res1
        
        x_res2 = Res2()(x, is_training)
        x = hk.Conv2D(output_channels=64, kernel_shape=1, stride=2)(x)
        x = x + x_res2
        
        x = hk.avg_pool(x, window_shape=3, strides=2, padding="VALID")
        x = hk.Flatten()(x)
        x = hk.Linear(len(classes))(x)
    
        return x
            

def _resnet(x, is_training=True):
    res_net = ResNet()
    return res_net(x, is_training)

resnet = hk.transform_with_state(_resnet)
resnet = hk.without_apply_rng(resnet)

#### init model

In [ ]:
rng = jax.random.PRNGKey(42)

x = jnp.ones((1, 32, 32, 3))
params, batch_states = resnet.init(rng, x)

print(f'param-shapes: \n{jax.tree_map(lambda x: x.shape, params)} \n')

print('batchnorm_states-shapes: ')
jax.tree_map(lambda x: x.shape, batch_states)

#### check forward pass

In [ ]:
x, y = next(iter(train_loader))
print('x.shape: ', x.shape)

preds, batch_states = resnet.apply(params, batch_states, x, is_training=False)
print('preds.shape: ', preds.shape)

#### define loss

In [ ]:
def loss(params, batch_states, x, y): 
  y_onehot =  jax.nn.one_hot(y, num_classes=10)
  y_hat, batch_states = resnet.apply(params, batch_states, x)
  return jnp.mean(optax.softmax_cross_entropy(y_hat, y_onehot)), batch_states

#### define optimizer and update function

In [ ]:
optimizer = optax.adam(learning_rate=1e-3) 
opt_state = optimizer.init(params)

@jax.jit
def update(params, batch_states, opt_state, x, y):
  grad, batch_states = jax.grad(loss, has_aux=True)(params, batch_states, x, y)
  updates, opt_state = optimizer.update(grad, opt_state, params)
  params = optax.apply_updates(params, updates)
  return params, batch_states, opt_state

#### train

In [ ]:
EPOCHS = 1
for epoch in range(EPOCHS): 
    for xs, ys in train_loader: 
        params, batch_states, opt_state = update(params, batch_states, opt_state, xs, ys)

#### test

In [ ]:
imgs, labels = next(iter(train_loader))
img = imgs[0]
im = np.expand_dims(img, axis=0)

print(img.shape)
print(im.shape)

im = np.expand_dims(img, axis=0)

prediction, batch_states = resnet.apply(params, batch_states, x=im, is_training=False)

pred = classes[np.argmax(prediction)]
label = classes[labels[0]]

print(f'predicted: {pred}, label: {label}')

import matplotlib.pyplot as plt
plt.imshow(img*0.5 + 0.5)
plt.axis('off')